In [ ]:
Exercice 4 : Exploration des Embeddings de Mots
Objectif : Comprendre l’utilisation des word embeddings pour la représentation des mots.
Tâches :
1.	Utiliser les embeddings de spaCy pour obtenir des vecteurs de mots.
2.	Visualiser les similarités entre différents mots à l'aide de PCA ou t-SNE.
3.	Expérimenter avec des mots similaires et observer les résultats des embeddings.


In [13]:
%pip install --upgrade --quiet  spacy

Note: you may need to restart the kernel to use updated packages.


In [18]:
import spacy

In [44]:
#1.	Utiliser les embeddings de spaCy pour obtenir des vecteurs de mots. Debut
nlp = spacy.load("fr_core_news_md")
text="Hello hello heLLO suis Inno Salut ello"
doc=nlp(text)
doc

Hello hello heLLO suis Inno Salut ello

In [40]:
print(doc[0] ,"Correspond à ",doc[0].vector[:30])
#1.	Utiliser les embeddings de spaCy pour obtenir des vecteurs de mots. Fin

Hello Correspond à  [ 0.63777   1.7131   -0.40801   0.6148    1.9123   -1.4067   -0.88647
 -1.8797    0.53062  -0.014785 -0.10438   0.2983    1.4436    2.4715
 -1.3017    0.32905  -0.80347  -1.9854   -0.036581 -1.2372   -0.076813
 -1.1983    2.3228   -0.22342   0.26895   0.76096   1.4202   -0.13129
  1.4403   -1.0978  ]


In [41]:
#Verifier s[1])i "Hello" similaire à "je"
doc[0].similarity(doc[1])

1.0000001192092896

In [42]:
doc.vector.shape

(300,)

In [45]:
n_chuck=list(doc.noun_chunks)
n_chuck

[Hello, heLLO]

In [56]:
#Verification de la distance entre thé et café
element_café=nlp("café")
element_thé=nlp("thé")


In [57]:
print(element_café.vector[:10])


[ 1.2429   0.42796 -2.3248  -0.14043 -0.45958  3.4885  -0.32466 -3.2992
 -1.1761  -1.0235 ]


In [58]:
print(element_thé.vector[:10])

[ 2.9725  -0.87587 -3.0441   1.1851   2.5585   8.1306   1.0907  -2.7831
 -0.73799 -3.1607 ]


In [59]:
element_café.similarity(element_thé)
#Ainsi l'ordinateur comprend que le café et le thé s'assemble un peu

0.7724714875221252

In [60]:
Avant de visualiser le mot incorporation, nous devons normalement d'abord entraîner le modèle. Cependant, l'incorporation de mots est très coûteuse en termes de calcul. Par conséquent, le modèle d'incorporation de mots pré-formé est normalement utilisé à la place. Il contient les jetons et leurs représentations vectorielles associées dans l'espace d'incorporation.

GloVe est un modèle d'intégration de mots pré-formé populaire développé par des chercheurs de Stanford à côté de word2vec qui est 
développé par Google. Dans cet article, l'incorporation de mots pré-entraînés GloVe est implémentée et vous pouvez la télécharger ici .
Pendant ce temps, nous pouvons utiliser la bibliothèque Gensim pour charger le modèle d'incorporation de
mots pré-entraîné. Vous pouvez installer la bibliothèque à l'aide de la commande pip comme suit.

SyntaxError: unterminated string literal (detected at line 1) (2535893236.py, line 1)

In [77]:
#2.	Visualiser les similarités entre différents mots à l'aide de PCA 
import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA
from gensim.models import KeyedVectors

def display_pca_scatterplot_3D(model, user_input=None, words=None, label=None, color_map=None, topn=5, sample=10):

    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
    
    word_vectors = np.array([model[w] for w in words])
    
    three_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:3]
    # For 2D, change the three_dim variable into something like two_dim like the following:
    # two_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:2]

    data = []
    count = 0
    
    for i in range (len(user_input)):

                trace = go.Scatter3d(
                    x = three_dim[count:count+topn,0], 
                    y = three_dim[count:count+topn,1],  
                    z = three_dim[count:count+topn,2],
                    text = words[count:count+topn],
                    name = user_input[i],
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 0.8,
                        'color': 2
                    }
       
                )
                
                # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable. Also, instead of using
                # variable three_dim, use the variable that we have declared earlier (e.g two_dim)
            
                data.append(trace)
                count = count+topn

    trace_input = go.Scatter3d(
                    x = three_dim[count:,0], 
                    y = three_dim[count:,1],  
                    z = three_dim[count:,2],
                    text = words[count:],
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 1,
                        'color': 'black'
                    }
                    )

    # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable.  Also, instead of using
    # variable three_dim, use the variable that we have declared earlier (e.g two_dim)
            
    data.append(trace_input)
    
# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(
        x=1,
        y=0.5,
        font=dict(
            family="Courier New",
            size=25,
            color="black"
        )),
        font = dict(
            family = " Courier New ",
            size = 15),
        autosize = False,
        width = 1000,
        height = 1000
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

model = KeyedVectors.load_word2vec_format(
    "GoogleNews-vectors-negative300.bin.gz", binary=True
)
display_pca_scatterplot_3D(PCA, "Café", "Thé")
